In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from torch import embedding
from nltk.corpus import stopwords
from nltk.corpus import wordnet 
import nltk
import numpy as np
from sklearn import preprocessing
import math

c:\Users\chang\anaconda3\envs\semantic\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
global q
q=0

In [3]:
from tracemalloc import start


def calculate(gcs,destination,timeframe,space,start_time=0,end=480): #所以gcs要是string

    
    df = pd.read_csv(gcs) 

    df['Start_time'] = pd.to_numeric(df['Start_time'])
    #df['end_time'] = pd.to_numeric(df['end_time'])
    global all_sentences001
    all_sentences001 = []
    t=start_time
    
    while t<end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            sentences.append(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        t+=space

    
    all_sentences001 = []
    stops = set(stopwords.words('english'))
    t=start_time
    while t <end:
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+timeframe))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
                #print(rows["Words"])
        all_sentences001.append(" ".join(sentences))
        #print(t)
        t += space 
    ''' 舊的sample step
    for t in range(0,480,0.5):
        sentences = []
        for index, rows in df.loc[(df["Start_time"]>=t) & (df["Start_time"]<=(t+10))].iterrows():
            if rows["Words"].lower() not in stops:
                sentences.append(rows["Words"].lower())
        all_sentences001.append(" ".join(sentences))
    '''    
    #nltk.download('omw-1.4')
    #nltk.download('wordnet')
    #nltk.download('stopwords')
    global localAncestors 
    localAncestors = [[]]*len(all_sentences001) #一個time frame之內ancestors' list
    global ancestors_depth
    ancestors_depth = []
    global pair_counts
    pair_counts = []
    stops = set(stopwords.words('english'))
    Timeframe_percentage = []  #每個timeframe的valid pair percentage的list
    for sentence_num in range(0,len(all_sentences001)): #現在第幾句
        wu_pulmer_sum = 0
        validPairs = 0
        the_sentence = str(all_sentences001[sentence_num]).split()
        #print(the_sentence)
        timeframe_pairs_count = 0
        for words_num in range(0, (len(the_sentence)-1)): #標定第一個字
            if len(wordnet.synsets(the_sentence[words_num])) > 0 :
                syns1 = wordnet.synsets(the_sentence[words_num])
                syns1_num = len(syns1)
            else:
                continue
            for words_num2 in range((words_num+1), len(the_sentence)): #標定第二個字
                
                hasAncestors = True
                if len(wordnet.synsets(the_sentence[words_num2])) > 0 :
                    syns2 = wordnet.synsets(the_sentence[words_num2])
                    syns2_num = len(syns2)
                    for i in range(0, syns1_num):  # 標定第一個synset
                        for u in range(0, syns2_num): #標定第二個synset
                            #print(the_sentence[words_num],"的",syns1[i]," ", the_sentence[words_num2],"的",syns2[u]," " ,syns1[i].wup_similarity(syns2[u]))
                            timeframe_pairs_count += 1
                            if syns1[i].lowest_common_hypernyms(syns2[u]) != []:
                                localAncestors[sentence_num].append(syns1[i].lowest_common_hypernyms(syns2[u])[0])
                                hasAncestors = True
                else:
                    continue
        #這邊是跑完一句話的處理
        if hasAncestors:
            validPairs += 1
        pair_counts.append(timeframe_pairs_count)
        #print(all_sentences_wu_sum)

        # 計算ancestors' depth
    print(localAncestors)
    for x in range(len(localAncestors)):
        localMin_depth = 0
        for u in range(0,len(localAncestors[x])):
            localMin_depth += localAncestors[x][u].min_depth() #一句話所有min depth加起來
        ancestors_depth.append((localMin_depth/len(localAncestors[x])))
    results_csv = pd.DataFrame(list(zip(pair_counts,ancestors_depth)), columns = ["pair_counts","ancestor_depth"])
    if q<10:
        results_csv.to_csv(f"data/Text/00{q}({destination}).csv")
    else:
        results_csv.to_csv(f"data/Text/0{q}({destination}).csv")
    print(timeframe_pairs_count)
    
    """ # for normalization
    wu_difference = np.array(wu_difference)
    SBERT = np.array(SBERT)
    #normalized_SBERT = preprocessing.normalize([SBERT])
    #normalized_wu_difference = preprocessing.normalize([wu_difference])


    min_max_scaler = preprocessing.MinMaxScaler()
    SBERT = np.reshape(SBERT,(479,1))
    wu_difference = np.reshape(wu_difference,(479,1))
    global normalized_SBERT
    global normalized_wu_difference
    normalized_SBERT  = min_max_scaler.fit_transform(SBERT)
    normalized_wu_difference  = min_max_scaler.fit_transform(wu_difference)
    """


In [4]:
#計算單一SUBJECT的切割
x=10
q=x
startt = input("enter start time: ")
endt= input("enter end time:")
cut_timeoffset = int(startt)
cut_timeend = int(endt)
calculate(f"data/Text/{q:0>3d}(done).csv",f"10pure_ancestor_test{startt}_{endt}",10,1,cut_timeoffset,cut_timeend)